#Librerias

In [3]:
from google.cloud import bigquery
#from google.colab import drive
#from google.colab import auth
import re
import os
import pandas as pd
#drive.mount('/content/drive')
#auth.authenticate_user()

project_id = 'iter-data-storage-pv-uat'
client = bigquery.Client(project_id)

In [4]:
path='Desarrollo/Replicacion UAT/replicacion_prod/sql_input/'
#path='/content/drive/MyDrive/Proyectos - IS/PROY - MAE/Desarrollo/Replicacion UAT/replicacion_prod/sql_input/'

#Codigos

In [15]:
def read_table_bq():
  sql = """
SELECT * FROM `iter-data-storage-pv.temp.MIGRACION_STORE_PROCEDURE`
WHERE nombre_store_procedure in (
'sp_inputfiles_producto_unidad_cuenta'
,'sp_inputfiles_ceco_unidad_cuenta'
,'sp_inputfiles_reaseguro_desgravamen'
,'sp_inputfiles_reaseguro_vida'
,'sp_inputfiles_tipo_cambio_ifrs17'
,'sp_inputfiles_siniestro_beneficiario'
,'sp_inputfiles_siniestro_desgravamen'
,'sp_inputfiles_siniestro_optativo'
,'sp_inputfiles_siniestro_proveedor'
,'sp_inputfiles_siniestro_vida'
,'sp_inputfiles_distribucion_costos_gastos'
,'sp_inputfiles_gastos_reales'
,'sp_bi_sbs_plnt_movimiento_contable_hist'
,'sp_bi_sbs_poliza_comprobante_hist'
,'sp_bi_sbs_plnt_pagos_compania_hist'
,'sp_bi_sbs_plnt_ifrs17_042'
,'sp_bi_sbs_normativo_ifrs17'
)"""
  df = pd.read_gbq(sql, project_id=project_id, dialect='standard')
  return df

In [18]:
def store_base(df):
  for index, row in df.iterrows():
    nombre_archivo = row['NOMBRE_STORE_PROCEDURE']
    contenido_archivo = row['DDL']
    with open(path + nombre_archivo+'.sql', 'w', newline='') as archivo:
      archivo.write(contenido_archivo)
  return 'Se exporto correctamente los DDL'

#Exporta un .sql por cada store procedure en la ruta

In [19]:
store_base(read_table_bq())

C:\Users\jberriot\AppData\Local\Temp\ipykernel_21988\3405439727.py:23: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  df = pd.read_gbq(sql, project_id=project_id, dialect='standard')


'Se exporto correctamente los DDL'

In [14]:
def buscar_tablas(sql_code):
    #table_pattern  = r'FROM\s+([^\s;]+)'
    pattern = r'(?<!PROCEDURE)\s+(?:FROM|JOIN|INTO|UPDATE|DELETE)\s+([^\s,;]+)'
    #pattern = r'(?:FROM|JOIN)\s+([A-Za-z0-9_]+)'
    matches = re.findall(pattern, sql_code, re.IGNORECASE)
    return matches

In [ ]:
archivos = os.listdir(path)
i=0
for archivo in archivos:
  print(archivo)
  with open(path + archivo,'r') as file:
    sql_code = file.read()
  data = buscar_tablas(sql_code)
  df = pd.DataFrame(data, columns=['NOMBRE_TABLA'])
  df['NOMBRE_STORE_PROCEDURE']=archivo
  if i==0:
    df_final = pd.DataFrame(df)
  else:
    df_final = pd.concat([df_final, df], ignore_index=True)
  df_final.head(3)
  i=i+1

In [ ]:
df_final.to_excel('resultado.xlsx')

In [ ]:
from google.cloud import bigquery
client = bigquery.Client()

#Procesa los DDL para replicar tablas con 0 registros

In [ ]:
query = """
SELECT DDL
FROM interseguro-data.temp.MIGRACION_TABLES
WHERE NOMBRE_TABLA NOT LIKE '%_landing'
AND NOMBRE_DATASET='goldenrecord_data'
AND TIPO='BASE TABLE'
"""
query_job = client.query(query)
results = query_job.result()

for row in results:
    ddl_statement = row['DDL']
    client.query(ddl_statement).result()

In [20]:
query = """
SELECT * FROM `iter-data-storage-pv.temp.MIGRACION_STORE_PROCEDURE`
WHERE nombre_store_procedure in (
'sp_inputfiles_producto_unidad_cuenta'
,'sp_inputfiles_ceco_unidad_cuenta'
,'sp_inputfiles_reaseguro_desgravamen'
,'sp_inputfiles_reaseguro_vida'
,'sp_inputfiles_tipo_cambio_ifrs17'
,'sp_inputfiles_siniestro_beneficiario'
,'sp_inputfiles_siniestro_desgravamen'
,'sp_inputfiles_siniestro_optativo'
,'sp_inputfiles_siniestro_proveedor'
,'sp_inputfiles_siniestro_vida'
,'sp_inputfiles_distribucion_costos_gastos'
,'sp_inputfiles_gastos_reales'
,'sp_bi_sbs_plnt_movimiento_contable_hist'
,'sp_bi_sbs_poliza_comprobante_hist'
,'sp_bi_sbs_plnt_pagos_compania_hist'
,'sp_bi_sbs_plnt_ifrs17_042'
,'sp_bi_sbs_normativo_ifrs17'
)
"""
query_job = client.query(query)
results = query_job.result()

for row in results:
    ddl_statement = row['DDL']
    client.query(ddl_statement).result()